In [4]:
### Library Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf 
import talib


In [6]:
### Load in Stock Data
tickers = ['GOOG', 'AAPL']
start_date = '2024-01-01'

def import_stock_data(tickers, start_date):
    # Check if tickers is a list or a single ticker string
    if isinstance(tickers, list):
        data = yf.download(tickers, start=start_date)[['Close', 'High', 'Low']]
    else:
        data = yf.download(tickers, start=start_date)[['Close', 'High', 'Low']]
        data = pd.DataFrame(data)

    # Reset index to make 'Date' a column instead of index
    data.reset_index(inplace=True)
    data['Date'] = pd.to_datetime(data['Date'])

    return data

stock_data = import_stock_data(tickers, start_date)
stock_data

[*********************100%%**********************]  2 of 2 completed


Price        Date       Close                    High                     Low  \
Ticker                   AAPL        GOOG        AAPL        GOOG        AAPL   
0      2024-01-02  185.639999  139.559998  188.440002  140.615005  183.889999   
1      2024-01-03  184.250000  140.360001  185.880005  141.089996  183.429993   
2      2024-01-04  181.910004  138.039993  183.089996  140.634995  180.880005   
3      2024-01-05  181.179993  137.389999  182.759995  138.809998  180.169998   
4      2024-01-08  185.559998  140.529999  185.600006  140.639999  181.500000   
..            ...         ...         ...         ...         ...         ...   
70     2024-04-12  176.550003  159.190002  178.360001  161.699997  174.210007   
71     2024-04-15  172.690002  156.330002  176.630005  160.830002  172.500000   
72     2024-04-16  169.380005  156.000000  173.759995  157.229996  168.270004   
73     2024-04-17  168.000000  156.880005  170.649994  158.681000  168.000000   
74     2024-04-18  167.039993  157.460007  168.639999  158.485001  166.550003   

Price               
Ticker        GOOG  
0       137.740005  
1       138.429993  
2       138.009995  
3       136.850006  
4       137.880005  
..             ...  
70      158.600006  
71      156.149994  
72      155.050003  
73      156.134995  
74      156.210007  

[75 rows x 7 columns]

In [7]:
### Calculate ADX
def calculate_ADX(data, window):
    # Calculate True Range (TR)
    data['TR'] = talib.TRANGE(data['High'], data['Low'], data['Close'])
    # Calculate the Directional Movement (DM)
    